The object of this program is loop through the fracfocus CSV files and add them to a SQLite database.

First we bring in the modules we need for this project

In [ ]:
import sqlite3                     #Bring in Sqlite for the database

from os import listdir             #Bring in OS to read all the files
from os.path import isfile, join  

import pandas as pd                #Bring in Pandas to work wiht the data

import pandas as pd                #Bring in Pandas to work wiht the data
import matplotlib.pyplot as plt    #Bring in matplotlib to run visualizations
import matplotlib.dates as mdates
import numpy as np                 #Bring in numpy for math operations

import time
from datetime import date
from datetime import datetime

Download the latest fracfocus data from http://fracfocus.org/data-download.  Download the csv files and save the unzipped files to your computer.  Then update 'mypath' below to that location.

The program then loops throught the file names in the folder then saves them to 2 lists.

In [ ]:
#mypath = "C:/Users/BWeaver/Google Drive/FracFocus/FracFocus"  #make sure the use back slashes or double forward slashes '\\'
mypath = '/Users/brianweaver/Documents/GitHub/Oil_and_Gas_Bid_Data/FF_Data'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]   #find files in the directory

FracFocusRegistry_files = list()
registryupload_files = list()

#loop through list of files and add them to the two lists
for file in onlyfiles:
    if file[:17] == 'FracFocusRegistry':
        FracFocusRegistry_files.append(file)
    elif file[:14] == 'registryupload':
        registryupload_files.append(file)

The program will then build the SQLite database if the file isn't there.  If the file is there then it will delete the FracFocusRegistry and registryupload tables.  

Then rebuild them with all the correct headers.  This was taken from the headers of the csv files

First we'll crreat a connection to the database

In [ ]:
conn = sqlite3.connect(mypath + '/FracFocus.sqlite')
cur = conn.cursor()

Next we'll make sure if the database is already made that the table is cleared out.

In [ ]:
cur.execute('''
DROP TABLE IF EXISTS FracFocusRegistry;
''')

Finally we'll rebuild the database

In [ ]:
cur.execute('''
CREATE TABLE FracFocusRegistry (
    id                        INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    UploadKey                 TEXT,
    JobStartDate              NUMERIC,
    JobEndDate                NUMERIC,
    APINumber                 TEXT,
    StateNumber               NUMERIC,
    CountyNumber              NUMERIC,
    OperatorName              TEXT,
    WellName                  TEXT,
    Latitude                  TEXT,
    Longitude                 TEXT,
    Projection                TEXT,
    TVD                       TEXT,
    TotalBaseWaterVolume      TEXT,
    TotalBaseNonWaterVolume   TEXT,
    StateName                 TEXT,
    CountyName                TEXT,
    FFVersion                 TEXT,
    FederalWell               TEXT,
    IndianWell                TEXT,
    Source                    TEXT,
    DTMOD                     TEXT,
    PurposeKey                TEXT,
    TradeName                 TEXT,
    Supplier                  TEXT,
    Purpose                   TEXT,
    SystemApproach            TEXT,
    IsWater                   TEXT,
    PurposePercentHFJob       TEXT,
    PurposeIngredientMSDS     TEXT,
    IngredientKey             TEXT,
    IngredientName            TEXT,
    CASNumber                 TEXT,
    PercentHighAdditive       TEXT,
    PercentHFJob              TEXT,
    IngredientComment         TEXT,
    IngredientMSDS            TEXT,
    MassIngredient            TEXT,
    ClaimantCompany           TEXT,
    DisclosureKey             TEXT
    
);
''')

Again, we'll drop the table for the registryupload to make sure it's cleared out for the new upload

In [ ]:
cur.execute('''
DROP TABLE IF EXISTS registryupload;
''')

Then rebuild it

In [ ]:
cur.execute('''
CREATE TABLE registryupload (
    id                        INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    pKey                      TEXT,
    JobStartDate              NUMERIC,
    JobEndDate                NUMERIC,
    APINumber                 TEXT,
    StateNumber               NUMERIC,
    CountyNumber              NUMERIC,
    OperatorName              TEXT,
    WellName                  TEXT,
    Latitude                  TEXT,
    Longitude                 TEXT,
    Projection                TEXT,
    TVD                       TEXT,
    TotalBaseWaterVolume      TEXT,
    TotalBaseNonWaterVolume   TEXT,
    StateName                 TEXT,
    CountyName                TEXT,
    FFVersion                 TEXT,
    FederalWell               TEXT,
    IndianWell                TEXT,
    Source                    TEXT,
    DTMOD                     TEXT
);
''')

Now that the database is built we'll start loading it with data from the csv files.  Starting wiht the FracFocusRegistry.

In [ ]:
for file in FracFocusRegistry_files:
    print(file)
    df = pd.read_csv(mypath + '/' + file)
    df.to_sql('FracFocusRegistry', conn, if_exists='append', index=False)
print('Complete!')

Next we upload the registryupload

In [ ]:
for file in registryupload_files:
    print(file)
    df = pd.read_csv(mypath + '/' + file)
    df.to_sql('registryupload', conn, if_exists='append', index=False)
print('Complete!')

Next close the connections

In [ ]:
cur.close()
conn.close()

Now the Database is setup

Let's see what the data looks like.  We'll use SQL to pull the Wyoming data.

In [ ]:
Wyoming_Data = pd.read_sql_query("""SELECT JobStartDate, JobEndDate, APINumber, StateNumber, CountyNumber, OperatorName, 
                      WellName, Latitude, Longitude, Projection, TVD, TotalBaseWaterVolume, 
                      TotalBaseNonWaterVolume, StateName, CountyName from registryupload
            WHERE     StateNumber = 5 """, conn)

In [ ]:
Wyoming_Data.head(10)

Next let's look at some time series information.  Becuase we just pull this data in we need to format it in order to filter off of it.  

First we'll format the date.  Even though it's readable to humans, the program needs it cleaned up.

In [ ]:
Wyoming_Data['JobStartDate'] = pd.to_datetime(Wyoming_Data['JobStartDate'])
Wyoming_Data['JobEndDate'] = pd.to_datetime(Wyoming_Data['JobEndDate'])

In [ ]:
Wyoming_Data.head(10)

In [ ]:
fig, ax = plt.subplots()                                    #Setup the plot
fig.set_size_inches(10, 7)                                  #Set the figure size
plt.style.use('fivethirtyeight')                            #Set the style, I'm a sucker for fivethirtyeight plots
#min_x = datetime.strptime('2010-06-01','%Y-%m-%d').date()   #set the Min date on the x-axis 
#max_x = datetime.strptime('2018-05-01','%Y-%m-%d').date()   #Set the Max date on the x-axis
ax.set_xlim(min_x, max_x)
ax.hist(Wyoming_Data['JobEndDate'].values, bins = 250)      #Set the Historgram paramaters
plt.show()

In [ ]:
Wyoming_Data['OperatorName'].value_counts()[:10].plot(kind='barh', figsize = (7,5))
plt.show()